In [3]:
# 使用 python-dotenv 加载 .env 文件
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.messages import HumanMessage, SystemMessage
from langchain.agents import create_agent, AgentState
from langchain.tools import tool, ToolRuntime

import os

# 加载当前目录下的 .env 文件
load_dotenv(override=True)

# 获取环境变量
api_key = os.getenv("API_KEY")
base_url = os.getenv("BASE_URL")
model_name = os.getenv("MODEL_NAME")


# 初始化模型
model = ChatOpenAI(
    base_url=base_url,
    model=model_name,
    api_key=api_key,
    temperature=1,
)
from typing import TypedDict


class State(AgentState):
    test: str


@tool
def get_weather(city: str, runtime: ToolRuntime) -> str:
    """获取指定城市的天气信息
    Args:
        city (str): 城市名称
    """
    # 这里可以集成实际的天气API调用
    test = runtime.state.get("test", "1")
    print(test)
    return f"{city}的天气是晴朗，温度25摄氏度。"


bi_agent = create_agent(model=model, tools=[get_weather], state_schema=State)

result = bi_agent.invoke(
    {
        "messages": [{"role": "user", "content": "上海天气如何"}],
        "test": 222,
    }
)

print(result)

222
{'messages': [HumanMessage(content='上海天气如何', additional_kwargs={}, response_metadata={}, id='806d9251-aba9-4fb5-8cc0-5ba6d3348019'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 269, 'total_tokens': 290, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-a1aa6354-aaaa-48da-8b98-31b87b453f9e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--0ca17d35-6a9d-4f83-a877-7ade660080bd-0', tool_calls=[{'name': 'get_weather', 'args': {'city': '上海'}, 'id': 'call_55491850000f41df81737f9c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 269, 'output_tokens': 21, 'total_tokens': 290, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}), ToolMessage(content='上海的天气是晴朗，温度25摄氏度。', name='get_weather', id='cb8bc735-ec11-478